In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", 
top_k = None, device = "mps")
classifier("I love this!")

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use mps:0
/Users/zack/Desktop/Projects/books-recommender/.venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.00041385198710486293},
  {'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'neutral', 'score': 0.00576459476724267},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'surprise', 'score': 0.00852868054062128}]]

In [4]:
prediction = classifier(books["description"][0].split("."))

sorted(prediction[0], key = lambda x: x["label"])


[{'label': 'anger', 'score': 0.009156353771686554},
 {'label': 'disgust', 'score': 0.0026284793857485056},
 {'label': 'fear', 'score': 0.06816215068101883},
 {'label': 'joy', 'score': 0.0479423962533474},
 {'label': 'neutral', 'score': 0.14038625359535217},
 {'label': 'sadness', 'score': 0.0021221640054136515},
 {'label': 'surprise', 'score': 0.7296021580696106}]

In [12]:
import numpy as np

emotional_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbn = []

emotion_scores = {label : [] for label in emotional_labels}
emotion_scores

{'anger': [],
 'disgust': [],
 'fear': [],
 'joy': [],
 'neutral': [],
 'sadness': [],
 'surprise': []}

In [10]:
def calculate_max_emotion_scores(predictions):
     per_emotion_scores = {label : [] for label in emotional_labels}
     for prediction in predictions:
          sorted_prediction = sorted(prediction, key=lambda x: x["label"])
          for index, label in enumerate(emotional_labels):
               per_emotion_scores[label].append(sorted_prediction[index]["score"])
     return {label : np.max(scores) for label, scores in per_emotion_scores.items()}



In [14]:
from tqdm import tqdm


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_score = calculate_max_emotion_scores(predictions)
    for label in emotional_labels:
        emotion_scores[label].append(max_score[label])

emotion_scores

100%|██████████| 5197/5197 [07:24<00:00, 11.69it/s]


{'anger': [0.06413352489471436,
  0.6126183867454529,
  0.06413352489471436,
  0.35148337483406067,
  0.08141220360994339,
  0.23222479224205017,
  0.5381850600242615,
  0.06413352489471436,
  0.3006710410118103,
  0.06413352489471436,
  0.06413352489471436,
  0.6115752458572388,
  0.10621538758277893,
  0.06413352489471436,
  0.07239237427711487,
  0.09586318582296371,
  0.162159264087677,
  0.06413352489471436,
  0.08193846791982651,
  0.22345983982086182,
  0.1071520745754242,
  0.06413352489471436,
  0.06413352489471436,
  0.06413352489471436,
  0.39636558294296265,
  0.06472660601139069,
  0.06413352489471436,
  0.06413352489471436,
  0.06413352489471436,
  0.30293378233909607,
  0.06683341413736343,
  0.06413352489471436,
  0.06413352489471436,
  0.16889655590057373,
  0.06413352489471436,
  0.06413352489471436,
  0.06413352489471436,
  0.07910337299108505,
  0.1185651570558548,
  0.06413352489471436,
  0.5636682510375977,
  0.9171825051307678,
  0.06835637241601944,
  0.22059889

In [15]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df['isbn13'] = isbn


emotions_df

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612618,0.348286,0.942528,0.704422,0.887940,0.111690,0.252544,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150722,0.360707,0.251881,0.732687,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255169,0.853722,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459270,0.759456,0.951104,0.368110,0.078765,9788185300535


In [16]:
books = pd.merge(books, emotions_df, on = "isbn13")

books.to_csv("books_with_emotions.csv", index=False)